# Lenne

We'll call our text-based digital assistant Lenne (for no actual reason)!

In [1]:
#needed packages along the way
import re
import random
import requests, json
import nltk

In [2]:
def weather(city):
    
    BASE_URL = "https://api.openweathermap.org/data/2.5/weather?"
    API_KEY = "put your API key here"
    CITY= city
    URL = BASE_URL + "q=" + CITY + "&appid=" + API_KEY + "&units=" + "metric"
    response = requests.get(URL)

    if response.status_code == 200:       
        
        data = response.json()
        main = data['main']
        temperature = main['temp']
        humidity = main['humidity']
        pressure = main['pressure']
        report = data['weather']
        feelslike = main['feels_like']
        #print(f"{CITY:-^30}")
        #print(f"Temperature in Celcius: {temperature}")
        #print(f"Feels Like in Celcius: {feelslike}")
        #print(f"Humidity: {humidity}")
        #print(f"Weather Report: {report[0]['description']}")
        print("The weather in "+CITY+" is "+report[0]['description']+" with a temperature of "+str(temperature)+" Degrees")
    else:
        print("Error in the HTTP request")

In [4]:
class Database:
    #return random weather based on location and time
    def get_weather(self, location, day):
        location=weather(location.lower())
        day=day.lower()
             
        if day=="monday":
            return "rainy"
        elif day=="tuesday" or day=="today":
            return "cloudy"
        elif day=="wednesday":
            return "cloudy with rain"
        elif day=="thursday":
            return "windy"
        elif day=="friday" or day=="tomorrow" or day=="saturday" or day=="sunday":
            return "snowy"
        
    #return random restaurant based on location and time
    def find_restaurant(self, price, food, area):
        if food.split()[0]=="persian":
            return {"name": "Zaffran", "address": "Stora Nygatan 3A"}
        elif food.split()[0]=="italian":
            return {"name": "La Cucina Italiana", "address": "Skånegatan 33"}
        elif food.split()[0]=="japanese":
            return {"name": "Sushi today", "address": "Aschebergsgatan 7"}
        elif food.split()[0]=="greek":
            return {"name": "Meat the Greek", "address": "Skanstorget 3"}
        elif food.split()[0]=="korean":
            return {"name": "Hello Monky", "address": "Linnégatan 52"}
        
    #return next bus/tram based on location, destination, time
    def find_transport(self, origin, destination, time):
        transportation_system=[{"type": "bus", "time": "12:35"}, {"type": "tram", "time": "18:47"},
                              {"type": "bus", "time": "1:23"}, {"type": "tram", "time": "22:30"},
                              {"type": "bus", "time": "19:19"}]
        random_number=random.randrange(len(transportation_system))
        return transportation_system[random_number]

In [5]:
class Frame:
    
    def __init__(self, database, domain, intent):
        self.database=database
        self.domain=domain
        self.intent=intent
        
    def fill_slots():
        pass
    
    def prompt():
        pass

In [6]:
class WeatherFrame(Frame):
    
    def __init__(self, database):
        super().__init__(database, domain="WEATHER-FORECAST", intent="WEATHER")
        self.location=None
        self.day=None
        
    def fill_slots(self, user_input):
        if not self.location:
            location_re="in (stockholm|gothenburg|malmö|bern|berlin|brussels|vienna|islamabad|rawalpindi|karachi)"
            temp=re.search(location_re, user_input)
            if temp:
                city=temp.group(0).split()[1:]
                city=' '.join(city)
                self.location=city.capitalize()
                
        if not self.day:
            day_re="on (sunday|monday|tuesday|wednesday|thursday|friday|saturday|today|tomorrow)"
            temp=re.search(day_re, user_input)
            if temp:
                self.day=temp.group(0).split()[1].capitalize()

    def prompt_location(self):
        return "I can help you with that. In which city?"
    
    def prompt_day(self):
        if self.location:
            return "On what day would you like to receive weather forecast in {}?".format(self.location)
        else:
            return "On what day would you like to receive weather forecast"
    
    def prompt(self):
        if not self.location:
            return self.prompt_location()
        
        if not self.day:
            return self.prompt_day()
        
        weather=self.database.get_weather(self.location, self.day)
        
    
    def is_filled(self):
        return self.location and self.day

In [7]:
class RestaurantFrame(Frame):
    
    def __init__(self, database):
        super().__init__(database, domain = "RESTAURANT", intent = "FIND-RESTAURANT")
        self.price=None
        self.food=None
        self.location=None
        
    def fill_slots(self, user_input):
        if not self.price:
            price_re="cheap|expensive|moderate"
            temp=re.search(price_re, user_input)
            if temp:
                self.price=temp.group(0).capitalize()
                
        if not self.food:
            food_re="(italian|persian|japanese|greek|korean) food"
            temp=re.search(food_re, user_input)
            if temp:
                self.food=temp.group(0)

        if not self.location:
            location_re="(in|near|around) (center|the south|the north|the east|the west)"
            temp=re.search(location_re, user_input)
            if temp:
                self.location=temp.group(0)
                
    def prompt_price(self):
        return "What price range are you interested in?"
        
    def prompt_food(self):
        return " I can help you with that. What type of food would you fancy?"
    
    def prompt_location(self):
        return "In what area would you like the restaurant to be?"
    
    def prompt(self):
        if not self.food:
            return self.prompt_food()
        
        if not self.location:
            return self.prompt_location()
        
        if not self.price:
            return self.prompt_price()
        
        restaurant=self.database.find_restaurant(self.price, self.food, self.location)
        respond="{} {} (located {}) can be found at the restaurant '{}'. The address is {}.".format(self.price, self.food, self.location, restaurant["name"], restaurant["address"])
        return respond

    def is_filled(self):
        return self.price and self.food and self.location

In [8]:
class TransportFrame(Frame):
    
    def __init__(self, database):
        super().__init__(database, domain="TRANSPORT", intent="FIND-TRAM-OR-BUS")
        self.location=None
        self.destination=None
        self.time=None
        
    def fill_slots(self, user_input):
        if not self.location:
            location_re="from [a-zA-Z]+"
            temp=re.search(location_re, user_input)
            if temp:
                self.location=temp.group(0).split()[1].capitalize()
                
        if not self.destination:
            destination_re="to [a-zA-Z]+"
            temp=re.search(destination_re, user_input)
            if temp:
                self.destination=temp.group(0).split()[1].capitalize()
                
        if not self.time:
            time_re="(at|in) (1[0-2]|0?[1-9])(:[0-5][0-9])?"
            temp=re.search(time_re, user_input)
            if temp:
                self.time=temp.group(0)

    def prompt_location(self):
        return "I can help you with that. What is your location?"
    
    def prompt_destination(self):
        return "Where would you like to go?"
    
    def prompt_time(self):
        return "When is your departure"
    
    def prompt(self):
        if not self.location:
            return self.prompt_location()
        
        if not self.destination:
            return self.prompt_destination()
        
        if not self.time:
            return self.prompt_time()
        
        transport = self.database.find_transport(self.location, self.destination, self.time)
        return "There is a {} going from {} to {} at {}.".format(transport["type"], self.location, self.destination, transport["time"])
    
    def is_filled(self):
        return self.location and self.destination and self.time

In [9]:
class Lenne:
    
    WEATHER=1
    RESTAURANT=2
    TRANSPORT=3
    
    WEATHER_KEYWORDS=["weather", "rainy", "sunny"]
    RESTAURANT_KEYWORDS=["restaurant", "hungry", "food", "eat"]
    TRANSPORT_KEYWORDS=["transport", "bus", "tram", "travel"]
    
    def __init__(self):
        self.db=Database()
        self.weather_frame=None
        self.restaurant_frame=None
        self.transport_frame=None
        self.current_intent=None
        
    def say(self, sentence):
        print("Lenne: ", sentence)
        
    def identify_intent(self, user_input):
        if any(keyword in user_input for keyword in self.WEATHER_KEYWORDS):
            self.current_intent=self.WEATHER
            if not self.weather_frame:
                self.weather_frame=WeatherFrame(self.db)
        elif any(keyword in user_input for keyword in self.RESTAURANT_KEYWORDS):
            self.current_intent=self.RESTAURANT
            if not self.restaurant_frame:
                self.restaurant_frame=RestaurantFrame(self.db)
        elif any(keyword in user_input for keyword in self.TRANSPORT_KEYWORDS):
            self.current_intent=self.TRANSPORT
            if not self.transport_frame:
                self.transport_frame=TransportFrame(self.db)

    def process_input(self, user_input):
        self.identify_intent(user_input)
        
        if self.current_intent==Lenne.WEATHER:
            self.weather_frame.fill_slots(user_input)
            self.say(self.weather_frame.prompt())
            if self.weather_frame.is_filled():
                self.weather_frame=None
                self.say("What else can I do for you?")
        elif self.current_intent==Lenne.RESTAURANT:
            self.restaurant_frame.fill_slots(user_input)
            self.say(self.restaurant_frame.prompt())
            if self.restaurant_frame.is_filled():
                self.restaurant_frame=None
                self.say("What else can I do for you?")
        elif self.current_intent==Lenne.TRANSPORT:
            self.transport_frame.fill_slots(user_input)
            self.say(self.transport_frame.prompt())
            if self.transport_frame.is_filled():
                self.transport_frame=None
                self.say("What else can I do for you?")
        else:
            self.say("Interesting, what can I do for you?")

In [10]:
assistant=Lenne()
assistant.say("Lenne at your service, what can I do for you?")
while (True):
    user_input=input()
    user_input=user_input.lower()
    if user_input=="nothing" or user_input=="bye" or user_input=="thanks" or user_input=="thank you":
        assistant.say("Have a nice day!")
        break
    else:
        assistant.process_input(user_input)

Lenne:  Lenne at your service, what can I do for you?
restraunts
Lenne:  Interesting, what can I do for you?
restaurant
Lenne:   I can help you with that. What type of food would you fancy?
japanese food please
Lenne:  In what area would you like the restaurant to be?
in center
Lenne:  What price range are you interested in?
cheap
Lenne:  Cheap japanese food (located in center) can be found at the restaurant 'Sushi today'. The address is Aschebergsgatan 7.
Lenne:  What else can I do for you?
weather
Lenne:  I can help you with that. In which city?
in gothenburg
Lenne:  On what day would you like to receive weather forecast in Gothenburg?
on tuesday
The weather in gothenburg is clear sky with a temperature of 3.62 Degrees
Lenne:  None
Lenne:  What else can I do for you?
bye
Lenne:  Have a nice day!
